# Notebook to plot Y chromosome diversity over time

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt
import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time
from scipy.spatial.distance import pdist

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
rcParams['font.sans-serif'] = ['Arial']
from matplotlib import gridspec
#plt.style.use('ggplot') #..../whitegrid.mplstyle  # Nice Plotting Style

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

from python.plot_pca import *  # Import functions needed for the PCA plotting
from hapsburg.PackagesSupport.sqrt_scale import SquareRootScale

compute-a-17-143.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 32
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


### Helper Functions

In [2]:
def filter_df_age(df, age_delta = 300, output=False):
    """Takes Dataframe df as Input, and filters to samples within age_delta of median age.
    Return filtered Dataframe and medium Age"""
    age_med = np.median(df["age"])
    idx = (df["age"]< age_med + age_delta) & (df["age"] > age_med - age_delta)
    df = df[idx].copy().reset_index(drop=False)
    if output:
        print(f"{np.sum(idx)}/{len(idx)} IIDs within {age_delta} y of median age {age_med}")
    return df, age_med

def get_y_counts(df, digits=3, col="Y_haplo"):
    """Get Y Chromosome counts from Dataframe df"""
    ys = df[col].str[:3]
    cts = ys.value_counts().values
    return cts

def simpson_di(x):
    """ Given a count vector, returns the Simpson Diversity Index
    """
    n = np.sum(x) # Sample Size
    h = np.sum(x*(x-1)) / (n*(n-1)) # Fraction of pairs are identiclal
    
    if h==0: ### Set minimimum homo-cutoff (one homo-pair):
        h = 2 / (n*(n-1))
    return 1 / h

def frac_max_haplo(x):
    """Given a count vector, return frequency of non most-common alleles"""
    f = np.max(x) / np.sum(x)
    return f

def create_ydiv_df(df, sites=[], col_loc="loc", method="simpson",
                   age_delta = 300, digits=3, min_m=5):
    """Take Meta Data as input, and for each site calculate
    the Simpson Index of Y chromosomes.
    method: simpson, frac_max_haplo"""
    data = []
    
    for s in sites:
        df_t = df[df[col_loc]==s]
        df_t, age = filter_df_age(df_t, age_delta=age_delta)
        m = len(df_t)
        
        if m >= min_m: # Only run full analysis if enough males
            y = get_y_counts(df_t)
            if method=="simpson":
                D = simpson_di(y)
            elif method=="frac_max_haplo":
                D = frac_max_haplo(y)
            else:
                raise RuntimeWarning("No fitting mode found.")
            data.append([s, age, m, D])
          
        else:
            continue

    df = pd.DataFrame(data)
    df.columns = ["loc", "age", "males", "D"]
    return df

def get_sub_df_region(df, region="", rec_col="region", loc_col="loc", min_n=5):
    """Get a Dataframe of Y haplogroup diversities per sites"""
    df_ib = df[df[rec_col].isin(region)]
    cts = df_ib[loc_col].value_counts()
    sites = cts[cts>=min_n].index.values
    df_y_it = create_ydiv_df(df, sites=sites)
    return df_y_it

def set_age_ydiv_df(df, site="", age=0,
                    site_col="loc", age_col="age"):
    """Set the Age of a Y Diversity Cluster"""
    idx = df[site_col]==site
    df.loc[idx, age_col]= age

def set_legends(ax, plots=[], legs=[], title="", loc="lower right"):
    """Set Legends in Panel Plots"""
    l1 = ax.legend(plots, legs, fontsize=11, loc=loc,
             title=title)
    
    l1.get_title().set_fontsize('13')
    l1.get_title().set_fontweight("bold")
    [lgd.set_color('white') for lgd in l1.legendHandles]
    [lgd.set_edgecolor('k') for lgd in l1.legendHandles]

### Load Meta and Y haplogroup Data

In [2]:
df_meta = pd.read_csv("/n/groups/reich/hringbauer/Data/v56.3.anno.haplogroups.csv") # Load Meta Data

min_snp = 100000 # Min SNP coverage for Y Call
age = [0,12000]
lat = [20,90]
lon = [-28, 180]
flag = ["_contam", "_dup"]

df_meta["study"]=df_meta["study"].fillna("missing") # Add Nan
idx = df_meta["n_cov_snp"]>min_snp
df=df_meta[idx].reset_index(drop=True)
print(f"Filtering to {np.sum(idx)}/{len(idx)} indiviuals with >{min_snp} SNPs.")
df["include"]=df["include_alt"].astype("int")

### Filtering based on Age
min_age=age[0]
idx = df["age"]>min_age
df=df[idx].reset_index(drop=True)
print(f"Filtering to {np.sum(idx)}/{len(idx)} inds >{min_age} BP.")

max_age = age[1]
idx = df["age"]<max_age
df = df[idx].reset_index(drop=True)
print(f"Filtering to {np.sum(idx)}/{len(idx)} inds <{max_age} BP.")

### Geographic Filtering
if (len(lat)>0) | (len(lon)>0):
    idx_lat = (lat[0] < df["lat"]) & (df["lat"] < lat[1])
    idx_lon = (lon[0] < df["lon"]) & (df["lon"] < lon[1])
    idx = (idx_lat & idx_lon)
    df=df[idx].reset_index(drop=True)
    print(f"Kept {np.sum(idx)}/{len(idx)} inds with matching lat/lon.")

### Flag tricky Indivdiuals
idx = df["clst"].str.contains("|".join(flag))
print(f"Kept {np.sum(~idx)}/{len(idx)} inds with good cluster labels.")
df=df[~idx].reset_index(drop=True)
df = df.sort_values(by="avg_cov_snp", ascending=False)
idx = df["Master ID"].duplicated()
print(f"Kept {np.sum(~idx)}/{len(idx)} unique Master IDs.")
df=df[~idx].reset_index(drop=True)

### Extract Males
idx= df["sex"]=="M"
print(f"Kept {np.sum(idx)}/{len(idx)} Males.")
df=df[idx].reset_index(drop=True)

### Flag Punic Individuals
df1 = pd.read_csv("./data/cluster_assignments_punic.v54.1i.tsv", sep="\t")
print(f"Extracted IIDs of {len(df1)} IIDs in Punic Project")
df_punic = pd.merge(df, df1, on="iid")
print(f"Merged to {len(df_punic)} Punic Males")

### Remove Romans
label_inc = ["Punic_Early", "Punic_Late", "Punic_Late2"]
df_punic = df_punic[df_punic["label"].isin(label_inc)]

#df_punic = df_punic[~df_punic["label"].str.contains("Roman")]
print(f"Filtered to {len(df_punic)} Punic Samples based on label")

### Remove Punics from generated Meta
df = df[~df["iid"].isin(df1["iid"])]
print(f"Filtered general Y to {len(df)} ancient, non Punic individuals")

### Go to published indivduals only
df =df[~df["study"].str.contains("Unpublished")]
print(f"Filtered to {len(df)} published ancient males")

Filtering to 29227/35545 indiviuals with >100000 SNPs.
Filtering to 22574/29227 inds >0 BP.
Filtering to 22448/22574 inds <12000 BP.
Kept 17910/22448 inds with matching lat/lon.
Kept 17707/17910 inds with good cluster labels.
Kept 16715/17707 unique Master IDs.
Kept 9262/16715 Males.
Extracted IIDs of 160 IIDs in Punic Project
Merged to 68 Punic Males
Filtered to 58 Punic Samples based on label
Filtered general Y to 9194 ancient, non Punic individuals
Filtered to 4059 published ancient males


### [Browse] See Punic Y Haplogroups

In [3]:
df_punic

,iid,Master ID,loc,lat,lon,age,region,study,clst,mean_cov,...,include,label_fine,location,label,clst_qpadm,cluster_geo,published,date range,direct date,SNPs
0,I24205,I24205,Kerkouane,36.947562,11.099143,2442.0,Tunisia,Unpublished (Harald Ilan Punic),Tunisia_Punic_oAfricaHigh,0.705550,...,1,Kerkouene,Kerkouene,Punic_Early,PunicKerkAfr,NorthAfrica,NaN,"723-404 calBCE (2415Â±20 BP, PSUAMS-11775)",y,846660
1,I24208,I24208,Kerkouane,36.947562,11.099143,2407.0,Tunisia,Unpublished (Harald Ilan Punic),Tunisia_Punic_Africa,0.702893,...,1,Kerkouene,Kerkouene,Punic_Early,PunicKerkAfrCline,NorthAfrica,NaN,"538-399 calBCE (2390Â±20 BP, PSUAMS-11777)",y,843472
2,I24555,I24555,"Sicily, Birgi",37.885000,12.475000,2250.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sicily_Punic_Late,0.701337,...,1,Birgi_Late,Birgi,Punic_Late,PunicLateEu,Sicily,NaN,"391-208 calBCE (2255Â±20 BP, PSUAMS-9227)",y,841605
3,I21859,I21859,"Sicily, Marsala, Lilybaeum, Necropoli Monumentale",37.885000,12.475000,2094.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sicily_Punic_Roman,0.667289,...,1,Lilybaeum_Late,Lilybaeum,Punic_Late2,PunicLateLevant,Sicily,NaN,"340-53 calBCE (2125Â±20 BP, PSUAMS-9206)",y,800747
4,I22118,I22118,"Sardinia, Tharros",39.873496,8.441024,2589.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sardinia_Punic_Early,0.668632,...,1,Tharros_Early,Tharros,Punic_Early,PunicCentralMedEarly,Sardinia,NaN,"767-519 calBCE (2480Â±20 BP, PSUAMS-9224)",y,802359
5,I22117,I22117,"Sardinia, Tharros",39.873496,8.441024,2582.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sardinia_Punic_Early,0.659316,...,1,Tharros_Early,Tharros,Punic_Early,PunicCentralMedEarly,Sardinia,NaN,"761-479 calBCE (2470Â±20 BP, PSUAMS-9808)",y,791179
7,I22114,I22114,"Sardinia, Tharros",39.873496,8.441024,2239.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sardinia_Punic_Late_oEurope,0.622697,...,1,Tharros_Late,Tharros,Punic_Late,PunicLateEu,Sardinia,NaN,"387-208 calBCE (2245Â±20 BP, PSUAMS-9222)",y,747236
8,I26932,I26932,"Málaga, Calle Mármoles",36.720200,-4.420300,2230.0,Spain,Unpublished (Harald Ilan Punic),Spain_Punic,0.685826,...,1,Malaga,Malaga,Punic_Late,PunicMalaga,Iberia,NaN,"383-201 calBCE (2225Â±25 BP, PSUAMS-9814)",y,822991
9,I22115,I22115,"Sardinia, Tharros",39.873496,8.441024,2645.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sardinia_Punic_Early,0.637866,...,1,Tharros_Early,Tharros,Punic_Early,PunicCentralMedEarly,Sardinia,NaN,"796-551 calBCE (2545Â±25 BP, PSUAMS-9223)",y,765439
10,I12844,I12844,"Sicily, Motya",37.535300,12.263700,2086.0,Italy,Unpublished (Harald Ilan Punic),Italy_Sicily_Punic_Roman,0.716190,...,1,Motya_Late2,Motya,Punic_Late2,PuncicMotya,Sicily,NaN,334-54 calBCE (2122Â±16 BP) [R_combine: (2130Â...,y,859428
